In [ ]:
# 

In [2]:
from datasets import load_dataset

# Load the SciTail dataset
scitail_dataset = load_dataset("scitail", "tsv_format")

# Accessing the train and validation sets
train_set = scitail_dataset['train']
validation_set = scitail_dataset['validation']

# Print some examples to see the data
print(train_set[0])
print(validation_set[0])

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 1304/1304 [00:00<00:00, 501317.36 examples/s]

{'premise': 'Pluto rotates once on its axis every 6.39 Earth days;', 'hypothesis': 'Earth rotates on its axis once times in one day.', 'label': 'neutral'}
{'premise': 'An introduction to atoms and elements, compounds, atomic structure and bonding, the molecule and chemical reactions.', 'hypothesis': 'Replace another in a molecule happens to atoms during a substitution reaction.', 'label': 'neutral'}
